In [3]:
from sklearn import datasets
iris = datasets.load_iris()
digits = datasets.load_digits()

In [6]:
digits.data.shape

(1797, 64)

In [8]:
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(digits.data[:-1], digits.target[:-1])

SVC(C=100.0, gamma=0.001)

In [16]:
clf.predict(digits.data[-1:])

array([8])

In [20]:
digits.data[-1:]

array([[ 0.,  0., 10., 14.,  8.,  1.,  0.,  0.,  0.,  2., 16., 14.,  6.,
         1.,  0.,  0.,  0.,  0., 15., 15.,  8., 15.,  0.,  0.,  0.,  0.,
         5., 16., 16., 10.,  0.,  0.,  0.,  0., 12., 15., 15., 12.,  0.,
         0.,  0.,  4., 16.,  6.,  4., 16.,  6.,  0.,  0.,  8., 16., 10.,
         8., 16.,  8.,  0.,  0.,  1.,  8., 12., 14., 12.,  1.,  0.]])

In [1]:
#import json
#json.loads('{"data": [0, 0, 10, 14, 8, 1, 0, 0, 0, 2, 16, 14, 6, 1, 0, 0, 0, 0, 15, 15, 8, 15, 0, 0, 0, 0, 5, 16, 16, 10, 0, 0, 0, 0, 12, 15, 15, 12, 0, 0, 0, 4, 16, 6, 4, 16, 6, 0, 0, 8, 16, 10, 8, 16, 8, 0, 0, 1, 8, 12, 14, 12, 1, 0]}')

In [13]:
from joblib import dump, load
dump(clf, 'smtest.joblib') 

['smtest.joblib']

In [3]:
#!pip install 'sagemaker[local]' --upgrade

In [ ]:
tar -cvpzf model.tar.gz model.joblib

In [4]:
import sagemaker

In [5]:
container="257446244580.dkr.ecr.eu-west-1.amazonaws.com/custom-images:neuralmagic"

In [1]:
import sagemaker 
import boto3
from sagemaker import get_execution_role 

role = get_execution_role()

In [2]:
from sagemaker.local import LocalSession

session = sagemaker.Session() #LocalSession()
#session.config = {'local': {'local_code': True}}

In [12]:
default_bucket = session.default_bucket()
print(default_bucket)

sagemaker-eu-west-1-257446244580


In [13]:
model_uri = session.upload_data(path="/home/ec2-user/SageMaker/SMTest/opt/ml/model/model.tar.gz", key_prefix="neural_magic")
print(model_uri)

s3://sagemaker-eu-west-1-257446244580/neural_magic/model.tar.gz


In [6]:
model = sagemaker.model.Model(image_uri="257446244580.dkr.ecr.eu-west-1.amazonaws.com/custom-images:neuralmagic",
                              name="neural-magic",
                              model_data="s3://sagemaker-eu-west-1-257446244580/neural_magic/model.tar.gz",
                              role=role,
                              sagemaker_session=session,
                              predictor_cls=sagemaker.Predictor
                             )

In [8]:
predictor = model.deploy(
    initial_instance_count=1, 
    instance_type='ml.m5.xlarge', 
    endpoint_name="neural-magic")

Using already existing model: neural-magic


---!

In [9]:
predictor

In [9]:
# from sagemaker import Predictor
# predictor = Predictor(endpoint_name=model.endpoint_name, )
# predictor

In [10]:
predictor.predict('{"data": [0, 0, 10, 14, 8, 1, 0, 0, 0, 2, 16, 14, 6, 1, 0, 0, 0, 0, 15, 15, 8, 15, 0, 0, 0, 0, 5, 16, 16, 10, 0, 0, 0, 0, 12, 15, 15, 12, 0, 0, 0, 4, 16, 6, 4, 16, 6, 0, 0, 8, 16, 10, 8, 16, 8, 0, 0, 1, 8, 12, 14, 12, 1, 0]}')

b'8'

In [12]:
predictor.delete_endpoint()